In [1]:
## model development and training
import torch
from torchvision.datasets import CIFAR10

from torch.utils.data import DataLoader
from torchvision import models
from torch import optim

import torch.nn as nn

In [2]:
import torch.optim as optim
import torch.nn.functional as F

In [3]:
## make a NN and feedforward network

class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3,6,5)

    self.pool = nn.MaxPool2d(2,2)

    self.conv2 = nn.Conv2d(6,16,5)

    self.fc1 = nn.Linear(16 *5*5,120)

    self.fc2 = nn.Linear(120,84)

    self.fc3 = nn.Linear(84,10)

  def forward(self, x):

    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1,16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)

    return x

In [4]:
net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [6]:
from torchvision import transforms

In [7]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


trainset = CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)


testset = CIFAR10(root='./data', train=False,
                                        download=False, transform=transform)
testLoader = DataLoader(trainset, batch_size=4,
                                          shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:10<00:00, 15.8MB/s]


In [8]:
## train the NN network

for epoch in range(10):

  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):

    input, labels = data

    ## zero the optimizer grad
    optimizer.zero_grad()

    ## forward + backward + optimize
    outputs = net(input)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()


    if i%2000 ==1999:
      print(f'{epoch+1}, {i+1} loss: {running_loss/2000}')
      running_loss = 0.0
print('Finished Training')

1, 2000 loss: 2.213680555999279
1, 4000 loss: 1.9156706486940385
1, 6000 loss: 1.696417014837265
1, 8000 loss: 1.6038744283914566
1, 10000 loss: 1.5296367170214653
1, 12000 loss: 1.4886816868185997
2, 2000 loss: 1.4036482996419073
2, 4000 loss: 1.3782630286216735
2, 6000 loss: 1.3565317768082024
2, 8000 loss: 1.3383126486465335
2, 10000 loss: 1.3068840001374482
2, 12000 loss: 1.2834400725215673
3, 2000 loss: 1.2061281855218113
3, 4000 loss: 1.1998794317021966
3, 6000 loss: 1.2146123760975898
3, 8000 loss: 1.1998662186712026
3, 10000 loss: 1.209570208242163
3, 12000 loss: 1.1792690735422076
4, 2000 loss: 1.1109321550913156
4, 4000 loss: 1.0983363323882223
4, 6000 loss: 1.1163717730306089
4, 8000 loss: 1.0980371537506581
4, 10000 loss: 1.108049239575863
4, 12000 loss: 1.1194958247821778
5, 2000 loss: 1.0208505357168616
5, 4000 loss: 1.0182847495810128
5, 6000 loss: 1.0270243784375488
5, 8000 loss: 1.0653964391341433
5, 10000 loss: 1.040215023238212
5, 12000 loss: 1.0583184533556924
6, 20

In [9]:
## train the NN network

for epoch in range(10):

  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):

    input, labels = data

    ## zero the optimizer grad
    optimizer.zero_grad()

    ## forward + backward + optimize
    outputs = net(input)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  net.eval()
  validation_loss = 0.0
  correct= 0
  total = 0
  with torch.no_grad():
    for data in testLoader:
      images, labels = data
      outputs = net(images)
      loss = criterion(outputs, labels)
      validation_loss += loss.item()

      _, preedicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      print(f'{epoch+1}, {i+1} loss: {running_loss/2000}')
      running_loss = 0.0
print('Finished VALIDATION')